In [11]:
from fyers_apiv3 import fyersModel
from datetime import datetime , timedelta
import pandas as pd
import pytz

In [2]:
client_id = "MACO3YJA7I-100"  # Replace with your client ID
secret_key = "N7SXUFQG91"  # Replace with your secret key
redirect_uri = "https://trade.fyers.in/api-login/redirect-uri/index.html"  # Replace with your redirect URI
response_type = "code" 
grant_type = "authorization_code"  
state = "sample_state"

In [4]:
# # # Generate Auth Code
# session = fyersModel.SessionModel(
#     client_id=client_id,
#     secret_key=secret_key,
#     redirect_uri=redirect_uri,
#     response_type=response_type
# )
# genrate_authcode = session.generate_authcode()
# print(genrate_authcode)

In [5]:
auth_code = "eyJ0eXAiOiJKV1QiLCJhbGciOiJIUzI1NiJ9.eyJpc3MiOiJhcGkubG9naW4uZnllcnMuaW4iLCJpYXQiOjE3MDMzNDczNTcsImV4cCI6MTcwMzM3NzM1NywibmJmIjoxNzAzMzQ2NzU3LCJhdWQiOiJbXCJ4OjBcIiwgXCJ4OjFcIiwgXCJ4OjJcIiwgXCJkOjFcIiwgXCJkOjJcIiwgXCJ4OjFcIiwgXCJ4OjBcIl0iLCJzdWIiOiJhdXRoX2NvZGUiLCJkaXNwbGF5X25hbWUiOiJZSjAwMTI5Iiwib21zIjoiSzEiLCJoc21fa2V5IjoiMjdlMDQ2OGE4MGJkYmVmMGQyYTg3M2MxMmNlN2YyZGEyMjA1NzQ0ZTIwYTg1MDI0YjhlMDA4NDciLCJub25jZSI6IiIsImFwcF9pZCI6Ik1BQ08zWUpBN0kiLCJ1dWlkIjoiYWRkMjNjMTYxOWFhNDdkNzkxNWRkZDU1ZjJkMTQ5NDUiLCJpcEFkZHIiOiIwLjAuMC4wIiwic2NvcGUiOiIifQ.RRUOp9Eqf_7E8okGp3AhmQOlRs-d6Fwu5Q5wllMfP7Y"

In [6]:
session = fyersModel.SessionModel(
    client_id=client_id,
    secret_key=secret_key, 
    redirect_uri=redirect_uri, 
    response_type=response_type, 
    grant_type=grant_type
)
session.set_token(auth_code)
response = session.generate_token()
access_token = response['access_token']

In [7]:
fyers = fyersModel.FyersModel(client_id=client_id, is_async=False, token=access_token, log_path="")

## ALL CODE

In [156]:
from ta.trend import EMAIndicator
import pandas_ta as pdta


def Entry_EQ(Symbol):
    data = {
                "symbol":Symbol,
                "resolution":"60",
                "date_format":"1",
                "range_from":(datetime.now() - timedelta(days=100)).strftime('%Y-%m-%d'),
                "range_to":datetime.now().strftime('%Y-%m-%d'),
                "cont_flag":"0"
            }
    row_data =  fyers.history(data=data)
    # print(row_data)
    df = pd.DataFrame.from_dict(row_data['candles'])
    columns_name = ['Datetime','Open','High','Low','Close','Volume']
    df.columns = columns_name
    df['Datetime'] = pd.to_datetime(df['Datetime'], unit='s')
    df['Datetime'] = df['Datetime'].dt.tz_localize(pytz.utc).dt.tz_convert('Asia/Kolkata')
    df['Datetime'] = df['Datetime'].dt.tz_localize(None)


    df['Pivot'] = (df['High'] + df['Low'] + df['Close']) / 3
    df['20EMA'] = EMAIndicator(close=df['Close'], window=20, fillna=False).ema_indicator()
    df['50EMA'] = EMAIndicator(close=df['Close'], window=50, fillna=False).ema_indicator()
    df['200EMA'] = EMAIndicator(close=df['Close'], window=200, fillna=False).ema_indicator()

    gap = 8
    df['Away_status'] = ((df['50EMA'] > df['Low'] - gap) & (df['50EMA'] < df['High'] + gap)) | ((df['200EMA'] > df['Low'] -gap) & (df['200EMA'] < df['High']+gap))

    super_trend = pdta.supertrend(high=df['High'], low=df['Low'], close=df['Close'], length=50, multiplier=4)
    df['Super_Trend'] = super_trend['SUPERTd_50_4.0']

    df['cross_status'] = 'No Signal'  # Default value

    cross_up_condition = (df['20EMA'] > df['50EMA']) & (df['20EMA'].shift(1) <= df['50EMA'].shift(1))
    cross_down_condition = (df['20EMA'] < df['50EMA']) & (df['20EMA'].shift(1) >= df['50EMA'].shift(1))
    
    df.loc[cross_up_condition, 'cross_status'] = 'Up'
    df.loc[cross_down_condition, 'cross_status'] = 'Down'

    df['Final_status'] = False  # Default value
    
    for i in range(10, len(df)):
        # Check if 'cross_status' is Up in any of the previous 10 days
        if any(df['cross_status'].iloc[i-10:i] == 'Up'):
            # Check if 'Super_Trend' has at least one '1' in the previous 10 days
            if 1 in df['Super_Trend'].iloc[i-10:i].tolist():
                df.at[i, 'Final_status'] = True

    return df.iloc[-1]['Final_status']

Symbol = "NSE:SBIN-EQ"
Entry_EQ(Symbol)

False

In [169]:
stock_list = ['BANKBARODA',
 'BEL',
 'BHEL',
 'MANAPPURAM',
 'MRPL',
 'NMDC',
 'NTPC',
 'NATIONALUM',
 'OIL',
 'PPLPHARMA',
 'POONAWALLA',
 'PFC',
 'TATAPOWER',
 'WIPRO']

# Multry Threding

In [178]:
import concurrent.futures
import pandas as pd

name = []  # You had a variable name conflict with the 'name' list and the 'name' module
status = []

def process_item(stock_name):  # Changed parameter name to avoid conflict
    Symbol = f"NSE:{stock_name}-EQ"
    result = Entry_EQ(Symbol)
    name.append(stock_name)
    status.append(result)


with concurrent.futures.ThreadPoolExecutor() as executor:
    executor.map(process_item, stock_list)

nj_df = pd.DataFrame({"Name": name, "Status": status})
nj_df[nj_df['Status']]


,Name,Status
0,MANAPPURAM,True
1,PPLPHARMA,True
